Prepare Cross Architecture Base Dataset

In [1]:
import pandas as pd

input_file = "/home/tommy/Project/PcodeBERT/dataset/csv/merged_adjusted_filtered.csv"
output_file = "/home/tommy/Project/PcodeBERT/dataset/csv/merged_filtered_final.csv"

try:
    df = pd.read_csv(input_file)

    families_to_exclude = ['dofloo', 'kaiji', 'mobidash', 'wroba']
    
    condition_to_exclude = (df['CPU'] == 'ARM') & (df['family'].isin(families_to_exclude))
    
    filtered_df = df[~condition_to_exclude]
    
    filtered_df.to_csv(output_file, index=False)
    
    print(f"資料過濾完成，已儲存至 {output_file}")
    print(f"原始筆數: {len(df)}")
    print(f"過濾後筆數: {len(filtered_df)}")

except FileNotFoundError:
    print(f"錯誤：找不到檔案 {input_file}")
except Exception as e:
    print(f"發生錯誤：{e}")

資料過濾完成，已儲存至 /home/tommy/Project/PcodeBERT/dataset/csv/merged_filtered_final.csv
原始筆數: 15622
過濾後筆數: 15291
